In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
# subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_with_path = ["" for i in range(2)]
subfolder_with_path[0] = main_directory + '/'+'01cluster_files'
subfolder_with_path[1] = main_directory + '/'+'zzz_input_and_reference_files'

In [5]:
###############################################################################################
# Importing information from "intensisties template" and "brain gubre map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])

# Reading "gubra_intensities_template.csv" file
gubra_intensities_template = pd.read_csv("gubra_intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "LabelAbrv": "string"})
# Reading "brain_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map_fixed.csv", 
                               dtype={"IDPath": "string", "LabelAbrv": "string",
                                      "raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [6]:
# Changing directory to the "01cluster_files" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "01cluster_files" folder
list = os.listdir(subfolder_with_path[0]) 
number_files = len(list)
number_cluster_files = number_files

In [7]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving file's name as array
    filename_segmented = file.split('_')

    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(7)
   

    # Adding a column to save hemispheres
    output['HEMISPHERE'] = ''
    output.loc[output.LabelID < 20000, 'HEMISPHERE'] = 'RIGHT'
    output.loc[output.LabelID > 20000, 'HEMISPHERE'] = 'LEFT'
        
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    #Adding a column with the value of the total of the found volume
    output['TOTAL_VOLUME'] = output['VOLUME'].sum()
    
    #Adding a column with the percentage of volume part form the total volume
    output['VOLUME_PERCENTAGE'] = (output['VOLUME'] /  output['TOTAL_VOLUME'] ) * 100
    
    #Adding a column with the cluster's name
    output['CLUSTER_NAME'] = 'CLUSTER_' + str(i + 1)
    
    #Adding columns with the first, second and third highest percentages
    first_three = output['VOLUME_PERCENTAGE'].nlargest(3).values
    output['FIRST_HIGHEST_VOLUME'] = output.loc[output['VOLUME_PERCENTAGE'] == first_three[0], 'raw'].item()
    output['FIRST_HIGHEST_VOL_%'] = first_three[0]
    output['SECOND_HIGHEST_VOLUME'] = output.loc[output['VOLUME_PERCENTAGE'] == first_three[1], 'raw'].item()
    output['SECOND_HIGHEST_VOL_%'] = first_three[1]
    output['THIRD_HIGHEST_VOLUME'] = output.loc[output['VOLUME_PERCENTAGE'] == first_three[2], 'raw'].item() 
    output['THIRD_HIGHEST_VOL_%'] = first_three[2]
  
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [8]:
#Extracting the just the first three biggest volummes for each cluster
sorted_volume_data = volume_data[['CLUSTER_NAME','FIRST_HIGHEST_VOLUME','FIRST_HIGHEST_VOL_%',
                                        'SECOND_HIGHEST_VOLUME','SECOND_HIGHEST_VOL_%',
                                        'THIRD_HIGHEST_VOLUME','THIRD_HIGHEST_VOL_%']].copy()

sorted_volume_data = sorted_volume_data.drop_duplicates(subset=['CLUSTER_NAME'])

In [10]:
# Changing directory to the main path
os.chdir(main_directory)

In [11]:
#creating the "OUTPUT_cluster_sorted_volume.csv" file
sorted_volume_data.to_csv('OUTPUT_cluster_sorted_volume.csv', index=False)

In [12]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

34  Files processed 
 
 Execution time: 1.65 seconds
